In [1]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os
import torch.nn.functional as F
import cv2
import numpy as np
import torch.utils.data as Data
from PIL import  Image,ImageDraw
import numpy as np


Batch_Size = 250
num_epochs = 200
learning_rate = 1e-5
samples = 20000

data1 = []
data2= []


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% --  Generate dataset
for i in range(samples):
    r = np.random.choice(np.arange(1, 4), p=[0.3,0.3,0.4])
    if r==1:
        img = Image.new('RGB', (28, 28), color = 'white')
        row,col,ch= np.shape(img)
        mean = 0
        var = 1
        sigma = var**0.5
        gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos1 = img + gauss_pos1
        im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
        draw = ImageDraw.Draw(im2_pos1)
        draw.rectangle([21, 12, 24, 15], fill='red', outline='red')
        data1.append(np.asarray(im2_pos1))
    elif r==2:
        img = Image.new('RGB', (28, 28), color = 'white')
        row,col,ch= np.shape(img)
        mean = 0
        var = 1
        sigma = var**0.5
        gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos1 = img + gauss_pos1
        im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
        draw = ImageDraw.Draw(im2_pos1)
        draw.rectangle([9,12,12,15],fill='red',outline='red')
        data1.append(np.asarray(im2_pos1))
    elif r==3:
        img = Image.new('RGB', (28, 28), color = 'white')
        row,col,ch= np.shape(img)
        mean = 0
        var = 1
        sigma = var**0.5
        gauss_pos1 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos1 = img + gauss_pos1
        im2_pos1 = Image.fromarray(noisy_pos1,'RGB')
        data1.append(np.asarray(im2_pos1))
        




for i in range(samples):
    
    r = np.random.choice(np.arange(1, 4), p=[0.3,0.3,0.4])

    if r==1:
        img = Image.new('RGB', (28, 28), color = 'white')
        row,col,ch= np.shape(img)
        mean = 0
        var = 1
        sigma = var**0.5
        gauss_pos2 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos2 = img + gauss_pos2
        im2_pos2 = Image.fromarray(noisy_pos2,'RGB')
        draw = ImageDraw.Draw(im2_pos2)
        draw.rectangle([21, 12, 24, 15], fill='red', outline='red')
        data2.append(np.asarray(im2_pos2))
    elif r==2:
        img = Image.new('RGB', (28, 28), color = 'white')
        row,col,ch= np.shape(img)
        mean = 0
        var = 1
        sigma = var**0.5
        gauss_pos2 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos2 = img + gauss_pos2
        im2_pos2 = Image.fromarray(noisy_pos2,'RGB')
        draw = ImageDraw.Draw(im2_pos2)
        draw.rectangle([9,12,12,15],fill='red',outline='red')
        data2.append(np.asarray(im2_pos2))
    elif r==3:
        img = Image.new('RGB', (28, 28), color = 'white')
        row,col,ch= np.shape(img)
        mean = 0
        var = 1
        sigma = var**0.5
        gauss_pos2 = np.random.normal(mean,sigma,(row,col,ch))
        noisy_pos2 = img + gauss_pos2
        im2_pos2 = Image.fromarray(noisy_pos2,'RGB')
        data2.append(np.asarray(im2_pos2))



#%%%%%%%%%%%%%%%%%%%%%%%%%%% Noisy data
img = Image.new('RGB', (28, 28), color = 'white')
row,col,ch= np.shape(img)
mean = 0
var = 1
sigma = var**0.5
gauss = np.random.normal(mean,sigma,(row,col,ch))
noisy = img + gauss
im2 = Image.fromarray(noisy,'RGB')
b = np.asarray(im2)
b = np.array(b, dtype=np.float32)
b=b[:,:,0]
b = torch.from_numpy(b)
print(np.shape(b))
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%       

#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Data Preparation

y0 = np.array(data1, dtype=np.float32)
y1 = np.array(data2, dtype=np.float32)
y0 = y0[:,:,:,0]
y1 = y1[:,:,:,0]

print(y0.shape)

x = torch.from_numpy(y0)
y = torch.from_numpy(y1)


torch_dataset = Data.TensorDataset(x,y)


loader = Data.DataLoader(

    dataset=torch_dataset,

    batch_size=Batch_Size,

    shuffle=True,

    num_workers=0,

)


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Defining NN

class AE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        self.encoder_hidden_layer = nn.Linear(
            in_features=kwargs["input_shape"], out_features=128
        )
        self.encoder_output_layer = nn.Linear(
            in_features=128, out_features=64
        )
        self.encoder_output_layer_1 = nn.Linear(
            in_features=64, out_features=32
        )
        self.encoder_output_layer_2 = nn.Linear(
            in_features=32, out_features=16
        )
        self.encoder_output_layer_3 = nn.Linear(
            in_features=16, out_features=8
        )
        self.encoder_output_layer_4 = nn.Linear(
            in_features=8, out_features=1
        )




    def forward(self, features):
        activation = self.encoder_hidden_layer(features)
        activation = torch.relu(activation)
        code = self.encoder_output_layer(activation)
        code = torch.relu(code)
        code = self.encoder_output_layer_1(code)
        code = torch.relu(code)
        code = self.encoder_output_layer_2(code)
        code = torch.relu(code)
        code = self.encoder_output_layer_3(code)
        code = torch.relu(code)
        code = self.encoder_output_layer_4(code)
        code = torch.sigmoid(code)
        return code

#  use gpu if available

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
model = AE(input_shape=784)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% - Loss Function

def h_score(fx, gy):
 
    fx = fx - fx.mean(0)

    gy = gy - gy.mean(0)

    Nsamples = fx.size(0)

    covf = torch.matmul((fx.t()), fx) / Nsamples

    covg = torch.matmul((gy.t()), (gy)) / Nsamples

    h = -2 * torch.mean((fx * gy).sum(1)) + (covf * covg).sum()

    return h



optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,weight_decay=1e-5)



for epoch in range(num_epochs):
    for x,y in loader:
        optimizer.zero_grad()
        # ===================forward=====================
        #loss = criterion(output1, img)
        x = x.view(-1, 784)
        y = y.view(-1, 784)
        loss = h_score(model(x),model(y))
        # ===================backward====================

        #optimizer_1.zero_grad()
        loss.backward()
        optimizer.step()
        #optimizer_1.step()
    # ===================log========================
    b = b.view(-1,784)
    print('epoch [{}/{}], loss:{:.4f}'.format(epoch+1, num_epochs, loss.item()))
    print(np.shape(x[0]))
    print("x: ", model(x[0]))
    print("y:", model(y[0]))
    print("b:", model(b))



torch.Size([28, 28])
(20000, 28, 28)
epoch [1/200], loss:0.0002
torch.Size([784])
x:  tensor([0.4051], grad_fn=<SigmoidBackward>)
y: tensor([0.5658], grad_fn=<SigmoidBackward>)
b: tensor([[0.5380]], grad_fn=<SigmoidBackward>)
epoch [2/200], loss:0.0007
torch.Size([784])
x:  tensor([0.8070], grad_fn=<SigmoidBackward>)
y: tensor([0.5560], grad_fn=<SigmoidBackward>)
b: tensor([[0.7432]], grad_fn=<SigmoidBackward>)
epoch [3/200], loss:0.0002
torch.Size([784])
x:  tensor([0.9821], grad_fn=<SigmoidBackward>)
y: tensor([0.9763], grad_fn=<SigmoidBackward>)
b: tensor([[0.9669]], grad_fn=<SigmoidBackward>)
epoch [4/200], loss:0.0000
torch.Size([784])
x:  tensor([0.9835], grad_fn=<SigmoidBackward>)
y: tensor([0.9860], grad_fn=<SigmoidBackward>)
b: tensor([[0.9787]], grad_fn=<SigmoidBackward>)
epoch [5/200], loss:-0.0001
torch.Size([784])
x:  tensor([0.9667], grad_fn=<SigmoidBackward>)
y: tensor([0.9343], grad_fn=<SigmoidBackward>)
b: tensor([[0.9743]], grad_fn=<SigmoidBackward>)
epoch [6/200], lo

epoch [44/200], loss:-0.0000
torch.Size([784])
x:  tensor([0.9990], grad_fn=<SigmoidBackward>)
y: tensor([0.9987], grad_fn=<SigmoidBackward>)
b: tensor([[0.9995]], grad_fn=<SigmoidBackward>)
epoch [45/200], loss:-0.0000
torch.Size([784])
x:  tensor([0.9992], grad_fn=<SigmoidBackward>)
y: tensor([0.9983], grad_fn=<SigmoidBackward>)
b: tensor([[0.9994]], grad_fn=<SigmoidBackward>)
epoch [46/200], loss:-0.0000
torch.Size([784])
x:  tensor([0.9986], grad_fn=<SigmoidBackward>)
y: tensor([0.9989], grad_fn=<SigmoidBackward>)
b: tensor([[0.9994]], grad_fn=<SigmoidBackward>)
epoch [47/200], loss:-0.0000
torch.Size([784])
x:  tensor([0.9983], grad_fn=<SigmoidBackward>)
y: tensor([0.9984], grad_fn=<SigmoidBackward>)
b: tensor([[0.9994]], grad_fn=<SigmoidBackward>)
epoch [48/200], loss:-0.0000
torch.Size([784])
x:  tensor([0.9989], grad_fn=<SigmoidBackward>)
y: tensor([0.9980], grad_fn=<SigmoidBackward>)
b: tensor([[0.9993]], grad_fn=<SigmoidBackward>)
epoch [49/200], loss:-0.0000
torch.Size([784]

epoch [87/200], loss:-0.0000
torch.Size([784])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [88/200], loss:-0.0000
torch.Size([784])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [89/200], loss:-0.0000
torch.Size([784])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [90/200], loss:-0.0000
torch.Size([784])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [91/200], loss:-0.0000
torch.Size([784])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [92/200], loss:-0.0000
torch.Size([784]

epoch [130/200], loss:-0.0000
torch.Size([784])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [131/200], loss:-0.0000
torch.Size([784])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [132/200], loss:-0.0000
torch.Size([784])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [133/200], loss:-0.0000
torch.Size([784])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [134/200], loss:-0.0000
torch.Size([784])
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
epoch [135/200], loss:-0.0000
torch.Size

epoch [173/200], loss:-0.0000
torch.Size([784])
x:  tensor([0.9994], grad_fn=<SigmoidBackward>)
y: tensor([0.9995], grad_fn=<SigmoidBackward>)
b: tensor([[0.9996]], grad_fn=<SigmoidBackward>)
epoch [174/200], loss:-0.0000
torch.Size([784])
x:  tensor([0.9994], grad_fn=<SigmoidBackward>)
y: tensor([0.9987], grad_fn=<SigmoidBackward>)
b: tensor([[0.9995]], grad_fn=<SigmoidBackward>)
epoch [175/200], loss:-0.0000
torch.Size([784])
x:  tensor([0.9990], grad_fn=<SigmoidBackward>)
y: tensor([0.9980], grad_fn=<SigmoidBackward>)
b: tensor([[0.9994]], grad_fn=<SigmoidBackward>)
epoch [176/200], loss:-0.0000
torch.Size([784])
x:  tensor([0.9984], grad_fn=<SigmoidBackward>)
y: tensor([0.9979], grad_fn=<SigmoidBackward>)
b: tensor([[0.9991]], grad_fn=<SigmoidBackward>)
epoch [177/200], loss:-0.0000
torch.Size([784])
x:  tensor([0.9980], grad_fn=<SigmoidBackward>)
y: tensor([0.9984], grad_fn=<SigmoidBackward>)
b: tensor([[0.9988]], grad_fn=<SigmoidBackward>)
epoch [178/200], loss:-0.0000
torch.Size

In [2]:
for i in range(20):
    img = Image.new('RGB', (28, 28), color = 'white')
    row,col,ch= np.shape(img)
    mean = 0
    var = 1
    sigma = var**0.5
    gauss = np.random.normal(mean,sigma,(row,col,ch))
    noisy = img + gauss
    im2_n = Image.fromarray(noisy,'RGB')
    b = np.asarray(im2_n)
    b = np.array(b, dtype=np.float32)
    b = torch.from_numpy(b)
    im2_test = Image.fromarray(noisy,'RGB')
    draw1 = ImageDraw.Draw(im2_test)
    draw1.rectangle([5, 9, 8, 12], fill='red', outline='red')
    z = np.asarray(im2_test)
    z = np.array(z, dtype=np.float32)
    z = torch.from_numpy(z)
    b=b[:,:,0]
    z=z[:,:,0]
    print("b:", model(b.view(-1,784)))
    print("z: ", model(z.view(-1,784)))
    print("x: ", model(x[i]))
    print("y:", model(y[i]))
    print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
    print("  ")
    p = Image.fromarray(data1[i])
    #p.show()
    q = Image.fromarray(data2[i])
    #q.show()

b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
z:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
x:  tensor([0.9999], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  
b: tensor([[0.9999]], grad_fn=<SigmoidBackward>)
z:  tensor([[0.9999]], grad_fn=<SigmoidBackward>)
x:  tensor([0.9999], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
z:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  
b: tensor([[1.0000]], grad_fn=<SigmoidBackward>)
z:  tensor([[1.0000]], grad_fn=<SigmoidBackward>)
x:  tensor([1.0000], grad_fn=<SigmoidBackward>)
y: tensor([1.0000], grad_fn=<SigmoidBackward>)
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%